Hello

In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import keras
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Activation,Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization

In [16]:
df=pd.read_csv('./dataset/label_binary.csv',dtype=str)
#df_test=pd.read_csv('./dataset/label_binary.csv',dtype=str) #for validation set

In [17]:
training_datagen = ImageDataGenerator(
rescale =1./255,
validation_split = 0.2)

training_set = training_datagen.flow_from_dataframe(
dataframe = df,
directory = './dataset/image/',
x_col = "file_name",
y_col = "label",
subset = "training",
seed = 67,
target_size = (256, 256), # Can change 512 to see effect of squishing data
batch_size = 30,
class_mode = 'categorical',
shuffle = True,
color_mode = 'grayscale')

validation_set = training_datagen.flow_from_dataframe(
dataframe = df,
directory = './dataset/image/',
x_col = "file_name",
y_col = "label",
subset = "validation",
seed = 67,
target_size = (256, 256), # Can change 512 to see effect of squishing data
batch_size = 30,
#class_mode = 'binary',
class_mode = 'categorical',
shuffle = True,
color_mode = 'grayscale') 

Found 2400 validated image filenames belonging to 2 classes.
Found 600 validated image filenames belonging to 2 classes.


In [18]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',input_shape=(256,256,1)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) #val to change
model.add(keras.layers.BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) #val to chang
model.add(keras.layers.BatchNormalization())



model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(keras.layers.BatchNormalization())
model.add(Dense(2, activation='softmax'))#2 for no. o classes at end
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 256, 256, 32)      320       
_________________________________________________________________
activation_14 (Activation)   (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_15 (Activation)   (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 127, 127, 32)     

In [20]:
history = model.fit(
x = training_set,
validation_data=validation_set, 
epochs=1)

80/80 [==============================] - 385s 5s/step - loss: 0.5516 - accuracy: 0.7883 - val_loss: 0.5776 - val_accuracy: 0.8067


In [ ]:
pd.DataFrame(history.history).plot()
plt.show()